# Procesado de los datasets inciales 
Primer vistazo a los datasets iniciales y preprocesado inicial para trabajar con ellos posteriormente. 

---

In [1]:
import numpy as np
import pandas as pd

## 1. Importar datos
- **clean_users**: genero, edad, categoria, regularidad, id
- **Prado_artworks_wikidata**: cuadros
- **Prado_emotions**: user, cuadro, emotion, polaridad
- **Prado_users**: id, edad, genero, pais

In [2]:
clean_users_data = "data/clean_users.csv"
cuadros_data = "data/Prado_artworks_wikidata.csv"
emotions_data = "data/Prado_emotions.csv"
prado_users_data = "data/Prado_users.csv"

clean_users  = pd.read_csv(clean_users_data)
cuadros = pd.read_csv(cuadros_data)
emotions = pd.read_csv(emotions_data)
prado_users = pd.read_csv(prado_users_data)

In [23]:
print(clean_users.shape)
clean_users.head(3)

(90, 6)


,user-gender,user-age,userCategory,museumVisits,timestamp,userId
0,Female,47.0,['Favourite-Painter'],['Rarely'],2021-04-16 12:23:31,1
1,Male,40.0,['Favourite-Painter'],['Sporadic'],2021-04-16 12:37:09,2
2,Female,34.0,['Amateur'],['Like-Museums'],2021-04-16 13:16:14,3


In [24]:
print(cuadros.shape)
cuadros.head(3)

(30, 13)


,ID,Category,Artist,Title,Year,Image URL,Painting Info URL,Artist Info URL,Location,wd:paintingID,wd:paintingURL,wd:artistID,wd:artistURL
0,57726f48edc2cb3880ba012f,Romanticism,Francisco Goya,Charles IV of Spain and his family,1800,https://uploads7.wikiart.org/images/francisco-...,https://www.wikiart.org/en/francisco-goya/char...,https://www.wikiart.org/en/francisco-goya,"Museo del Prado, Madrid, Spain",Q920030,http://www.wikidata.org/entity/Q920030,Q5432,http://www.wikidata.org/entity/Q5432
1,57726e34edc2cb3880b62340,High Renaissance,Correggio,Noli Me Tangere,1534,https://uploads1.wikiart.org/images/correggio/...,https://www.wikiart.org/en/correggio/noli-me-t...,https://www.wikiart.org/en/correggio,"Museo del Prado, Madrid, Spain",Q124362,http://www.wikidata.org/entity/Q124362,Q8457,http://www.wikidata.org/entity/Q8457
2,57726f56edc2cb3880ba7233,Romanticism,Francisco Goya,"Summer, or The Harvest",1786,https://uploads3.wikiart.org/images/francisco-...,https://www.wikiart.org/en/francisco-goya/summ...,https://www.wikiart.org/en/francisco-goya,"Museo del Prado, Madrid, Spain",Q5849818,http://www.wikidata.org/entity/Q5849818,Q5432,http://www.wikidata.org/entity/Q5432


In [25]:
print(emotions.shape)
emotions.head(3)

(1759, 4)


,userId,artworkId,emotion,Polarity
0,6,5772700eedc2cb3880bc82ce,trust,positive
1,20,5772700eedc2cb3880bc82ce,fear,negative
2,20,5772700eedc2cb3880bc82ce,sadness,negative


In [26]:
print(prado_users.shape)
prado_users.head(3)

(171, 4)


,userId,age,gender,country
0,1,55-64,male,Spain
1,2,25-34,female,Spain
2,3,18-24,female,Spain


## 2. Procesar datos

#### - Reducir paises a regiones:
> Ya que el dataset de prado_users tiene pais vamos a utilizar el dataset "countries of the world" para identificar las regiones de las que proceden

In [3]:
countries_data = "data/countries_of_the_world.csv"
countries = pd.read_csv(countries_data)
countries = countries.filter(items = ['Country', 'Region'])

countries.head()

,Country,Region
0,Afghanistan,ASIA (EX. NEAR EAST)
1,Albania,EASTERN EUROPE
2,Algeria,NORTHERN AFRICA
3,American Samoa,OCEANIA
4,Andorra,WESTERN EUROPE


In [4]:
# # Eliminar el espacio en blanco despues de cada nombre de pais
# for i in range(0, countries.shape[0] - 1):
#     countries.loc[i] = countries.loc[i].replace(countries.loc[i]['Country'], countries.loc[i]['Country'][:-1])

In [5]:
# countries.to_csv("data/countries_of_the_world.csv")

In [6]:
# ## Sustituiremos cada pais de procedencia de prado_users por un numero q corresponda con su region
# # Ver cuantas regiones hay
# num_regions = 0
# for i in countries['Region'].unique():
#     num_regions += 1
#     countries = countries.replace(i,num_regions)

In [7]:
# Formatear los nombres de USA y Argentina
prado_users = prado_users.replace('USA','United States')
prado_users = prado_users.replace('Argenitina','Argentina')

In [8]:
# Reemplazar los paises por el numero de regiones
for country in prado_users['country'].unique():
    if country == 'Other':
        prado_users = prado_users.replace(country,0) #sustituir por 0 los usuarios que no sabemos de donde son
    else:
        prado_users = prado_users.replace(country,countries.loc[countries['Country'] == country]['Region'].to_list()[0])

In [11]:
countries.loc[countries['Country'] == 'United States']['Region'].to_list()[0]

'NORTHERN AMERICA                   '

In [10]:
prado_users.head()

,userId,age,gender,country
0,1,55-64,male,WESTERN EUROPE
1,2,25-34,female,WESTERN EUROPE
2,3,18-24,female,WESTERN EUROPE
3,4,18-24,female,WESTERN EUROPE
4,5,35-44,male,WESTERN EUROPE


#### - Formatear rangos de edades:
> Asignar a cada rango de edad un número, igual que hicimos en el paso anterior con las regiones

In [39]:
# Formatear los <12 y >70
prado_users = prado_users.replace('<12', '0-12')
prado_users = prado_users.replace('>70', '70-100')

In [42]:
# Calcularel numero correspondiente a cada rango de edad
age_index = 1

for age_range in prado_users.sort_values(by=['age'])['age'].unique():
    prado_users = prado_users.replace(age_range, age_index)
    age_index += 1
age_index -=1

In [43]:
prado_users.head()

,userId,age,gender,country
0,1,7,male,5
1,2,4,female,5
2,3,3,female,5
3,4,3,female,5
4,5,5,male,5


#### - Formatear género:
> Cambiar el género por un número

In [44]:
prado_users = prado_users.replace('female', 0)
prado_users = prado_users.replace('male', 1)

In [45]:
prado_users.head()

,userId,age,gender,country
0,1,7,1,5
1,2,4,0,5
2,3,3,0,5
3,4,3,0,5
4,5,5,1,5


## 3. Normalizar datos


In [56]:
from sklearn.preprocessing import StandardScaler

In [57]:
users_scaled = prado_users.copy()
users_scaled.drop('userId', axis=1, inplace=True) # No queremos normalizar el id (no es necesario para el clustering)

In [58]:
scaler = StandardScaler(with_mean=False, with_std=True)

users_scaled = pd.DataFrame(data=scaler.fit_transform(users_scaled), columns=users_scaled.columns.to_list())

In [59]:
users_scaled

,age,gender,country
0,4.107464,2.021723,1.787405
1,2.347123,0.000000,1.787405
2,1.760342,0.000000,1.787405
3,1.760342,0.000000,1.787405
4,2.933903,2.021723,1.787405
...,...,...,...
166,3.520684,2.021723,1.787405
167,3.520684,2.021723,1.787405
168,2.347123,0.000000,1.787405
169,3.520684,2.021723,1.787405


In [60]:
users_scaled.insert(0, 'userId', prado_users.userId)

In [61]:
users_scaled

,userId,age,gender,country
0,1,4.107464,2.021723,1.787405
1,2,2.347123,0.000000,1.787405
2,3,1.760342,0.000000,1.787405
3,4,1.760342,0.000000,1.787405
4,5,2.933903,2.021723,1.787405
...,...,...,...,...
166,298,3.520684,2.021723,1.787405
167,299,3.520684,2.021723,1.787405
168,300,2.347123,0.000000,1.787405
169,301,3.520684,2.021723,1.787405


In [64]:
users_scaled.to_csv("data/Prado_users_scaled.csv", index= 0)